In [20]:
# Import all the dependencies
import string
from tqdm.auto import tqdm
import json
import os

We can verify whether a `StringB` is a valid suffix of `StringA` by implementing a logic like this :
`StringA[len(StringA)-len(StringB):] == StringB`

Which as same as leveraging the below algorithm:

```Python
def GetLCSLength(stringA, stringB, stringALength, stringBlength):
    if (stringALength == 0 or stringBlength == 0):
        return 0
    if (stringA[stringALength-1] == stringB[stringBlength-1]):
        return 1 + GetLCSLength(stringA, stringB, stringALength-1, stringBlength-1)
    else:
        return max(GetLCSLength(stringA, stringB, stringALength, stringBlength-1), GetLCSLength(stringA, stringB, stringALength-1, stringBlength))

def IsSuffix(stringA, stringB,stringALength, stringBlength):
    return True if (GetLCSLength(stringA, stringB, stringALength, stringBlength)== stringBlength) else False
```


In [19]:
# Set file location
dir = os.path.dirname(os.getcwd())
fileName = os.path.join(dir, 'bengaliai-stemmingbengaliwords','Datasets','bengaliWords.tsv')


# Filtered dataset

## Using a dictionary to avoid duplication entry and for O(c) access
## This dictionary will hold the first iteration subsequences as the keys and the extensions as values
firstIteration = {}


# Noise cancellation
# Remove all the non-bengali characters
with open(fileName,"r") as rawFile:
    lines = rawFile.readlines()
    for line in tqdm(lines):
        if line[0] != "#"and line[0] not in string.ascii_letters and line.strip() :
            firstIteration[line.split()[0] ]= []

# Append origin words to the trailing strings' list
# Bruteforce approach is required because we are considering every word as an trailing subsequence
# This is the 1st iteration
for stringA in tqdm(firstIteration.keys()):
    for stringB in firstIteration.keys():
        if stringA != stringB and stringB[len(stringB)-len(stringA):] == stringA:
            firstIteration[stringA].append(stringB[:len(stringB)-len(stringA)])



In [24]:
## Don't stem for alphabets
for key,value in tqdm(firstIteration.copy().items()):
    if len(key) <= 2 or len(value) == 0:
        firstIteration.pop(key)

## save the output as json
fileName = os.path.join(dir, 'bengaliai-stemmingbengaliwords','Output','firstIteration.json')
with open(fileName, 'w') as jsonObject:
    json.dump(firstIteration, jsonObject, ensure_ascii=False)

In [25]:
#Inversing the dataset for O(c) access

inversedfirstIteration = {}
for key in tqdm(firstIteration.keys()):
    for value in firstIteration[key]:
        if value not in inversedfirstIteration:
            inversedfirstIteration[value] = [key]
        else:
            inversedfirstIteration[value].append(key)

fileName = os.path.join(dir, 'bengaliai-stemmingbengaliwords','Output','inversedfirstIteration.json')
with open(fileName, 'w') as jsonObject:
    json.dump(firstIteration, jsonObject, ensure_ascii=False)

        

In [ ]:
inversedfirstIteration

3